##### Import .env File

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
assert(os.getenv("OPENAI_API_KEY")is not None)

### Interfaces

#### LLM Interface

In [51]:
from langchain_openai.llms import OpenAI
model = OpenAI(model="babbage-002", temperature=0.1, max_tokens=3)
model.invoke("The sky is")

' the limit for'

#### Chat Interface

In [52]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=3)
prompt = [HumanMessage("The sky is")]
model.invoke(prompt)

AIMessage(content=' blue and clear', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 10, 'total_tokens': 13, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOQop0vIxwv1IxYzMTxcCWLEAGdk6', 'finish_reason': 'length', 'logprobs': None}, id='run-72778ebf-75cb-4b1e-94cf-f81d212bc89e-0', usage_metadata={'input_tokens': 10, 'output_tokens': 3, 'total_tokens': 13, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Instead of single prompt string, chat models make use of different types of chat message interfaces associated with each role mentioned  
1. Human Message  
    User
2. AI Message  
    AI
3. System Message  
    Instructions for AI to follow
4. Chat Message  
    A msg for allowing arbitary setting of role

### Prompt Template Interface

In [55]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template("""
    Answer the following question with a single word.
    Context: {context}
    Question: {question}
    Answer:
    """)
prompt = template.invoke({
    "context": "Paris is the one of the cites of France, along with Lyon, Marseille, and Nice but others are not the capital.",
    "question": "What is the capital of France?"
})
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=3)
response = model.invoke(prompt)
response

AIMessage(content='Paris', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 60, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOQxTO1B5vcBBFiA3yj30NMYBw795', 'finish_reason': 'stop', 'logprobs': None}, id='run-adf2d36b-ec7b-4764-924a-fc0644ea749c-0', usage_metadata={'input_tokens': 60, 'output_tokens': 2, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [95]:
# Similarly Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system',"Answer the following question with a single word."),
    ('human',"Context: {context}"),
    ('human',"Question: {question}"),
    ('human',"Answer:")
])
prompt = template.invoke({
    "context": "Paris is the one of the cites of France, along with Lyon, Marseille, and Nice but others are not the capital.",
    "question": "What is the capital of France?"
})
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=3)
response = model.invoke(prompt)
response

AIMessage(content='Paris', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 66, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BOSJKRPAGp5UnSl6iv4mN2BmGeHVk', 'finish_reason': 'stop', 'logprobs': None}, id='run-ebbabd1e-2dff-4ab0-a6be-447bdd5e603d-0', usage_metadata={'input_tokens': 66, 'output_tokens': 2, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Structured Output

In [59]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel
class AnswerWithJustification(BaseModel):
    '''Answer to the user's question along with the justification''' #doc_string; access it like AnswerWithJustification.__doc__
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''The justification for the answer'''

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_structured = model.with_structured_output(AnswerWithJustification)
model_structured.invoke("""What weights more, a pound of feathers or a pound of bricks?""")
#Output is a object of AnswerWithJustification with answer and justification attributes

d:\GitHub\GenAI\venv-genAI\Lib\site-packages\langchain_openai\chat_models\base.py:1660: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


AnswerWithJustification(answer='A pound of feathers and a pound of bricks weigh the same: one pound.', justification='Weight is a measure of mass, and one pound is equal to one pound regardless of the material. Therefore, a pound of feathers weighs the same as a pound of bricks.')

Pydantic library is used for validating data against schemas  
Schema is sent to LLM in JSONSchema object and sent to LLM.  
Schema is also used to validate the output returned by LLM to see validity of output

#### Output Parsers

In [61]:
# 2 uses. Provide extra information to the model and get the answer in a structured format.
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
items = parser.invoke("Red Car, Blue Car, Green Car")
items

['Red Car', 'Blue Car', 'Green Car']

### Assembling Many Pieces of an LLM Application

#### Runnable Interfaces

In [65]:
#1. Invoke 
#      single input to output
#2. Batch 
#      multiple inputs to multiple outputs
#3. Stream
#      output from single input as its produced (async)
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
completion = model.invoke('Hi There!')
print(completion)
completions = model.invoke(['Hi There!', 'Bye'])
print(completions)
for token in model.stream('Hi There!'):
    print(token)


content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BORUT4MicydvfFadJp3yX8b7zSTuS', 'finish_reason': 'stop', 'logprobs': None} id='run-c6bc3cca-1a35-48aa-aec0-dbf376ce506e-0' usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content="Hello! If you have any questions or need assistance, feel free to ask. If you're leaving, take care!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'p

#### Imperative Composition

##### Basic

In [87]:
# fancy name for writing code in component-wise fashion
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

template = ChatPromptTemplate.from_messages([
    ('system',"You are a helpful assistant."),
    ('human',"Question: {question}")
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def chatbot(values):
    prompt = template.invoke(values)
    response = model.invoke(prompt)
    return response

print(chatbot({"question": "Best  GPT model for learning purposes?"}).content)

For learning purposes, the best GPT model to use often depends on your specific needs and goals. Here are a few options to consider:

1. **GPT-3.5**: This model is widely used and offers a good balance of performance and accessibility. It can handle a variety of tasks, including text generation, summarization, and question answering. It's a great choice for general learning and experimentation.

2. **GPT-4**: If you have access to GPT-4, it is more advanced than GPT-3.5 and can provide better responses, especially for complex queries. It is particularly useful for in-depth learning and understanding nuanced topics.

3. **OpenAI's Playground**: This is an interactive platform where you can experiment with different models, including GPT-3.5 and GPT-4. It allows you to test prompts and see how the models respond, which can be very educational.

4. **Fine-tuned Models**: Depending on your area of interest, you might find fine-tuned versions of GPT models that are specialized for certain t

##### Streaming Imperative Composition

In [88]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

template = ChatPromptTemplate.from_messages([
    ('system',"You are a helpful assistant."),
    ('human',"Question: {question}")
])

@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token 

for part in chatbot.stream({"question": "Best  GPT model for learning purposes?"}):
    print(part)

content='' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content='For' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' learning' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' purposes' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=',' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' the' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' best' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' GPT' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' model' additional_kwargs={} response_metadata={} id='run-7f1e1a8f-1d3f-4cf7-a1a1-c3b4feb30e11'
content=' to' additional_kwargs={} response_metadata={} id='run-7f1e

##### Async Imperative Compostion

In [89]:
@chain 
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await model.ainvoke(prompt)

await chatbot.ainvoke({"question": "Best  GPT model for learning purposes?"})

AIMessage(content="For learning purposes, the best GPT model to use often depends on your specific needs and goals. Here are a few options to consider:\n\n1. **GPT-3.5**: This model is widely used and offers a good balance of performance and accessibility. It can handle a variety of tasks, including text generation, summarization, and question answering. It's a great choice for general learning and experimentation.\n\n2. **GPT-4**: If you have access to GPT-4, it is more advanced than GPT-3.5 and can provide better responses, especially for complex queries. It is particularly useful for in-depth learning and understanding nuanced topics.\n\n3. **OpenAI's Playground**: This is an interactive platform where you can experiment with different models, including GPT-3.5 and GPT-4. It allows you to test prompts and see how the models respond, which can be very educational.\n\n4. **Fine-tuned Models**: Depending on your area of interest, you might find fine-tuned versions of GPT models that ar

#### Declarative Composition

In [90]:
#LCEL: Declarative language for composing Langchain components
#Optimised for Parallel execution, Streaming, tracing, and async support

from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system',"You are a helpful assistant."),
    ('human',"Question: {question}")
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chatbot = template | model
chatbot.invoke({"question": "Best  GPT model for learning purposes?"})

AIMessage(content="For learning purposes, the best GPT model to use often depends on your specific needs and goals. Here are a few options to consider:\n\n1. **GPT-3.5**: This model is widely used and offers a good balance of performance and accessibility. It can handle a variety of tasks, including text generation, summarization, and question answering. It's a great choice for general learning and experimentation.\n\n2. **GPT-4**: If you have access to GPT-4, it is more advanced than GPT-3.5 and can provide better performance in terms of understanding context, generating coherent text, and handling complex queries. It’s ideal for more in-depth learning and applications that require higher accuracy.\n\n3. **OpenAI's Playground**: This is a user-friendly interface that allows you to experiment with different models, including GPT-3.5 and GPT-4. It’s a great way to learn how to interact with the models and understand their capabilities.\n\n4. **Fine-tuned Models**: Depending on your area

In [92]:
#also either use following
await chatbot.ainvoke({"question": "Best  GPT model for learning purposes?"})

AIMessage(content="For learning purposes, the best GPT model to use often depends on your specific needs and goals. Here are a few options to consider:\n\n1. **GPT-3.5**: This model is widely used and offers a good balance of performance and accessibility. It can handle a variety of tasks, including text generation, summarization, and question answering. It's a great choice for general learning and experimentation.\n\n2. **GPT-4**: If you have access to GPT-4, it is more advanced than GPT-3.5 and can provide better responses, especially for complex queries. It is particularly useful for in-depth learning and understanding nuanced topics.\n\n3. **OpenAI's Playground**: This is an interactive platform where you can experiment with different models, including GPT-3.5 and GPT-4. It allows you to test prompts and see how the models respond, which can be very educational.\n\n4. **Fine-tuned Models**: Depending on your area of interest, you might find fine-tuned versions of GPT models that ar

In [93]:
for part in chatbot.stream({"question": "Best  GPT model for learning purposes?"}):
    print(part)

content='' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content='For' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' learning' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' purposes' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=',' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' the' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' best' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' GPT' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' model' additional_kwargs={} response_metadata={} id='run-19e12f96-9a56-47b0-b029-8caf0c7e5d74'
content=' to' additional_kwargs={} response_metadata={} id='run-19e1